In [89]:
import pandas as pd
import numpy as np
import wget
import os

In [86]:
# pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=b6bcf64dfe09dfbc37479bbcce901cb0cc964e3b0b0c365343750c9b8c270daf
  Stored in directory: /Users/chenrong/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [57]:
df = pd.read_csv('./resource/phenotypes_202304020100.csv', sep=';')

In [58]:
df.head()
df.shape

(6674, 719)

In [59]:
avail = df[df["Colour Blindness"]!='-']

In [60]:
avail['Colour Blindness'].unique()

array(['False', 'Not color blind', 'Not colour blind', 'not color blind',
       'Red-green colour blind', 'Mild red-green colour blind',
       'Mild blue-black colour blind',
       'low light blue green colour blindness',
       'Low light blue green colour blindness',
       'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',
       'Tritanomaly',
       'Trouble distinguishing purples, dark blue, violets. ',
       'Moderate to Strong Protanopia',
       'Not color blind, but my parent is color blind', nan, 'N/a', 'Yes',
       'slight blue, purple'], dtype=object)

## Two ways to seperate phenotypes:

1. Yes/No Color-Blindness

Yes: ['Red-green colour blind', 'Mild red-green colour blind', 'low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'Moderate to Strong Protanopia', 'Yes', 'slight blue, purple', 'Trouble distinguishing purples, dark blue, violets. ']

No: ['False', 'Not color blind', 'Not colour blind', 'not color blind', 'Not color blind, but my parent is color blind', 'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',]

2. Type of Color-Blindess

Red-green: ['Red-green colour blind', 'Mild red-green colour blind', 'Moderate to Strong Protanopia']

Blue-yellow: ['low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'slight blue, purple', 'Trouble distinguishing purples, dark blue, violets. ']

In [61]:
avail['user_id'] #there are individuals with multiple files

0           1
1           1
2           1
3           1
5           8
        ...  
6592    11641
6593    11643
6622    11698
6624    11700
6650    11753
Name: user_id, Length: 785, dtype: int64

In [62]:
avail.shape

(785, 719)

In [63]:
avail

,user_id,genotype_filename,date_of_birth,chrom_sex,openhumans_name,Eye color,Lactose intolerance,Handedness,white skin,Ability to find a bug in openSNP,...,Dementia Resistance,Short Stature,Root resorption of tooth,Congenital adrenal hyperplasia (CAH),Difficulty reaching climax/ orgasming with partner,Thin Skin,Excessive sweating from heat (Eccrine sweat glands),Emotional sweating (apocrine sweat glands),Cold-Induced Sweating,"Epicanthic Fold, Hooded Eyes"
0,1,1.23andme.9,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
1,1,1.23andme.2995,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
2,1,1.23andme-exome-vcf.3302,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
3,1,1.23andme.5299,1985,XY,-,blue-green,lactose-tolerant,left-handed,Caucasian,Only creating bugs,...,-,-,-,-,-,-,-,-,-,-
5,8,8.23andme.2,rather not say,XX,-,Brown,lactose-intolerant,left-handed,True,c-c-combo breaker!,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,11641,11641.ftdna-illumina.9569,1990,XY,Daiten,Green-gray,-,"Right-handed dominant, left-handed for eating",-,-,...,-,-,-,-,-,-,-,-,-,-
6593,11643,11643.23andme.9571,rather not say,rather not say,-,Brown,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6622,11698,11698.23andme.9613,1997,XY,takiwatanga,Brown-green,Partially lactose intolerant - surfaces with a...,Left-handed,Caucasian,Positive,...,-,-,Yes,suspected diagnosis,Almost Impossible,-,Yes,High,Yes,-
6624,11700,11700.ancestry.9615,rather not say,rather not say,-,-,-,Right-handed,Caucasian,-,...,-,-,-,-,-,-,-,-,-,-


In [64]:
cb = avail[['user_id', 'genotype_filename', 'date_of_birth', 'chrom_sex', 'Colour Blindness']]
cb = cb.rename(columns={'Colour Blindness':'raw_colourblindness'})
cb = cb.dropna()
cb = cb[cb['raw_colourblindness']!='N/a']
cb.groupby('raw_colourblindness', axis=0).count()['user_id']

raw_colourblindness
Carrier (eg: my brother and my son have colour blindness)                                                      4
False                                                                                                        124
Low light blue green colour blindness                                                                          4
Mild blue-black colour blind                                                                                   4
Mild red-green colour blind                                                                                   24
Moderate to Strong Protanopia                                                                                  1
No                                                                                                            10
Not color blind                                                                                              303
Not color blind, but my parent is color blind                               

In [80]:
yes  = {key:True for key in ['Red-green colour blind', 'Mild red-green colour blind', 'low light blue green colour blindness', 'Low light blue green colour blindness', 'Tritanomaly', 'Moderate to Strong Protanopia']}
no = {key:False for key in ['False', 'Not color blind', 'Not colour blind', 'not color blind', 'Not color blind, but my parent is color blind', 'trouble distinguishing between dark blue & black (not color blindness), dad had red-green color blindness',
       'No', 'carrier (eg: my brother and my son have colour blindness)',
       'Carrier (eg: my brother and my son have colour blindness)',
       'Not colour blind, but difficulty telling red/orange/pink apart sometimes',]}
yes.update(no)
cb['CBDiscrete'] = cb['raw_colourblindness'].map(yes)
cb

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
0,1,1.23andme.9,1985,XY,False,False
1,1,1.23andme.2995,1985,XY,False,False
2,1,1.23andme-exome-vcf.3302,1985,XY,False,False
3,1,1.23andme.5299,1985,XY,False,False
5,8,8.23andme.2,rather not say,XX,False,False
...,...,...,...,...,...,...
6592,11641,11641.ftdna-illumina.9569,1990,XY,False,False
6593,11643,11643.23andme.9571,rather not say,rather not say,Red-green colour blind,True
6622,11698,11698.23andme.9613,1997,XY,False,False
6624,11700,11700.ancestry.9615,rather not say,rather not say,Not colour blind,False


In [84]:
counts = cb.groupby('user_id').count()
counts

,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
user_id,,,,,
1,4,4,4,4,4
8,1,1,1,1,1
10,2,2,2,2,2
11,1,1,1,1,1
14,1,1,1,1,1
...,...,...,...,...,...
11641,1,1,1,1,1
11643,1,1,1,1,1
11698,1,1,1,1,1


In [73]:
colorBlind = cb[cb['CBDiscrete']==True]

In [77]:
cb.groupby('CBDiscrete').count()

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness
CBDiscrete,,,,,
False,720,720,720,720,720
True,59,59,59,59,59


In [81]:
cb[cb['user_id']==1]

,user_id,genotype_filename,date_of_birth,chrom_sex,raw_colourblindness,CBDiscrete
0,1,1.23andme.9,1985,XY,False,False
1,1,1.23andme.2995,1985,XY,False,False
2,1,1.23andme-exome-vcf.3302,1985,XY,False,False
3,1,1.23andme.5299,1985,XY,False,False


In [88]:
urlHead = "https://opencravat.org/data/open-snp/"